In [1]:
# import basic packages
%run SetUp.ipynb

Configuration.

In [2]:
## Configuration
Title = "Exp_MNIST"
device = "cuda:1" if torch.cuda.is_available() else "cpu"

# create directory
output_path = f"./Results/{Title}"
create_directory(output_path)

# data setting
DATA = "MNIST"
img_size = 28
n = 60000 # number of data 
d = 1*img_size*img_size # dimension of data
output_class = 10 # output dimension // trainning multi single-layer networks
y_true = 10 # true label
y_false = 1 # false label

# load data 
X, Y = data_load(DATA)
X = X.to(device) # input X is of shape n x d 
Y = (y_false + (y_true-y_false)*(Y == torch.arange(10)).float()).to(device) # label Y is one hot

# UserWarning occurs, due to the torch version and torchvision version.

/home/user/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [3]:
net = ReLU_Net(output_class=output_class)
net.module[0].weight = nn.Parameter(torch.rand(output_class, d)*0.0000001)
net.to(device)


# Optimization setting
Epochs = 1000*1000
lr = 0.005
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=lr)
folder_name = output_path + '/runs/' + datetime.datetime.now().strftime("%B%d_%H_%M_%S")
writer = SummaryWriter(folder_name)
# Save CFG file
f = open(f"./{folder_name}/CFG.text", 'w')
f.write(
f"""
    This is a CFG file.

    # dataset
    {DATA}
    img_size = {img_size}
    n = {n} # number of data
    d = {d}
    y_true = {y_true}
    y_false = {y_false}
    num_class = {output_class}

    # optimization
    Epochs = {Epochs}
    lr = {lr}
"""
)
f.close()

In [4]:
# loss_tr = np.empty(0)
print("Start Training")
time.sleep(1)

log_period = 10
for epoch in tqdm(range(Epochs)) :
    loss = criterion(net(X), Y)
    loss.backward()
    optimizer.step()
    net.zero_grad()
    
    if epoch % log_period == 0 :
#         loss_tr = np.append(loss_tr, loss.item())
        writer.add_scalar("ReLU/Loss", loss.item(), epoch+1)
        writer.add_scalar("ReLU/# of SVs", (net(X)>0).prod(axis=1).sum().item(), epoch+1)
        writer.add_scalar("ReLU/# of nSVs", (net(X)==0).prod(axis=1).sum().item(), epoch+1)

Start Training


  2%|▏         | 20650/1000000 [00:44<35:33, 459.00it/s]


KeyboardInterrupt: 

In [ ]:
# save non-support vectors by images
# nSVs = X[(net(X)==0).squeeze()]
nSVs_index = (net(X)==0).prod(axis=1).nonzero()[:,0]
for index in nSVs_index:
    plt.imshow(X[index].view(28,28).cpu().detach(), cmap='gray')
    plt.savefig(folder_name+f"/{index}.png")
    plt.close()
print(f"There are {len(nSVs_index)} deactivated data")

In [ ]:
# save
torch.save(net.state_dict(), f"./{folder_name}/net.pt")